<a href="https://colab.research.google.com/github/xmks-colab/SmartCutter/blob/main/SmartStitch_para_celular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@title SmartStitch { display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os
from google.colab import files
import shutil
import sys

def run_command(cmd, silent=True):
    """Executa comando silenciosamente"""
    try:
        if silent:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)
        else:
            result = subprocess.run(cmd, shell=True, check=True)
        return True, result
    except subprocess.CalledProcessError as e:
        return False, e

def install_smartstitch():
    print("Preparando...")

    # Verifica se o diretório já existe
    if os.path.exists('SmartStitch'):
        print("SmartStitch já está instalado. Ignorando...")
        os.chdir('SmartStitch')
        print("Finalizado.")
        return

    # Clone do repositório
    success, _ = run_command("git clone https://github.com/MechTechnology/SmartStitch.git")
    if not success:
        print("Erro ao clonar o repositório.")
        return

    # Mudança para o diretório
    os.chdir('SmartStitch')

    # Instalação das dependências
    packages = ["numpy", "pillow", "natsort", "psd-tools"]
    for package in packages:
        success, _ = run_command(f"pip install {package}")
        if not success:
            print(f"Erro ao instalar {package}.")
            return

    print("Finalizado.")

# Executar instalação
install_smartstitch()

# --- Criar pasta input automaticamente ---
def create_input_folder():
    input_path = "/content/SmartStitch/input"
    os.makedirs(input_path, exist_ok=True)
    return input_path

# --- pasta imutavel ---
input_folder = "input"

split_height = widgets.IntText(
    value=5000,
    description="Split H:",
    layout=widgets.Layout(width="40%")
)

output_format = widgets.Dropdown(
    options=[".png", ".jpg", ".webp", ".bmp", ".psd", ".tiff", ".tga"],
    value=".png",
    description="Formato:"
)

custom_width = widgets.IntText(
    value=0,
    description="Largura:",
    layout=widgets.Layout(width="30%")
)

detection_type = widgets.Dropdown(
    options=["pixel", "none"],
    value="pixel",
    description="Detecção:"
)

sensitivity = widgets.IntSlider(
    value=90, min=0, max=100, step=1,
    description="Sensib:"
)

quality = widgets.IntSlider(
    value=100, min=1, max=100, step=1,
    description="Qualidade:"
)

ignorable_pixels = widgets.IntText(
    value=5,
    description="Ign Pixels:"
)

scan_line = widgets.IntSlider(
    value=5, min=1, max=100, step=1,
    description="Scan Line:"
)

# --- Checkbox para zip e download ---
zip_download = widgets.Checkbox(
    value=True,
    description="Zipar e Baixar",
    indent=False
)

# --- 2. Saída e Função de Upload ---
output = widgets.Output()

def upload_images(b):
    output.clear_output()
    with output:
        input_path = '/content/SmartStitch/input'
        print(f"📁 Usando pasta input: {input_path}")

        print("📤 Selecione as imagens para upload...")

        original_cwd = os.getcwd()
        try:
            os.chdir(input_path)
            uploaded = files.upload()
        finally:
            os.chdir(original_cwd)

        if uploaded:
            print(f"📥 {len(uploaded)} arquivo(s) salvo(s) diretamente na pasta input...")
            for filename in uploaded.keys():
                dst_path = f"{input_path}/{filename}"
                print(f"✅ {filename} → {dst_path}")

            print(f"🎉 Upload concluído! {len(uploaded)} imagem(s) salvas em {input_path}")
        else:
            print("❌ Nenhum arquivo foi selecionado")

def run_smartstitch(b):
    output.clear_output()

    # Garantir que a pasta existe antes de executar
    input_path = create_input_folder()

    with output:
        # Verificar se há imagens na pasta
        if not os.path.exists(input_path):
            print("❌ Pasta input não existe!")
            return

        images = [f for f in os.listdir(input_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp', '.tiff', '.tga', '.psd'))]
        if not images:
            print("❌ Nenhuma imagem encontrada na pasta input!")
            print("💡 Use o botão 'Upload' para enviar imagens primeiro.")
            return

        print(f"📸 Encontradas {len(images)} imagem(s): {', '.join(images[:3])}{'...' if len(images) > 3 else ''}")

    cmd = [
        "python", "SmartStitchConsole.py",
        "-i", input_folder,
        "-sh", str(split_height.value),
        "-t", output_format.value
    ]

    if custom_width.value > 0:
        cmd += ["-cw", str(custom_width.value)]
    if detection_type.value:
        cmd += ["-dt", detection_type.value]
    if sensitivity.value:
        cmd += ["-s", str(sensitivity.value)]
    if quality.value:
        cmd += ["-lq", str(quality.value)]
    if ignorable_pixels.value:
        cmd += ["-ip", str(ignorable_pixels.value)]
    if scan_line.value:
        cmd += ["-sl", str(scan_line.value)]

    with output:
        print(f"\n🚀 Executando: {' '.join(cmd)}")
        print("-" * 50)
        process = subprocess.run(cmd, text=True, capture_output=True)

        if process.stdout.strip():
            print("📋 Output:")
            print(process.stdout)

        if process.stderr.strip():
            print("⚠️ Avisos/Erros:")
            print(process.stderr)

        print("-" * 50)
        print("✅ Processamento concluído!")

        # Zip e download se checkbox ativado
        if zip_download.value:
            output_folder = "/content/SmartStitch/input [stitched]"
            if os.path.exists(output_folder):
                zip_path = "/content/SmartStitch/stitched_results"
                shutil.make_archive(zip_path, 'zip', output_folder)
                zip_file = f"{zip_path}.zip"
                print(f"📦 Zip criado: {zip_file}")
                print("⬇️ Iniciando download...")
                files.download(zip_file)
            else:
                print(f"❌ Pasta de saída não encontrada: {output_folder}")

def clear_input_folder(b):
    output.clear_output()
    input_path = '/content/SmartStitch/input'
    with output:
        if os.path.exists(input_path):
            for filename in os.listdir(input_path):
                file_path = os.path.join(input_path, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"❌ Erro ao limpar {file_path}: {e}")
            print("✅ Pasta de entrada limpa!")
        else:
            print("❌ Pasta de entrada não existe!")

def clear_output_folder(b):
    output.clear_output()
    output_folder = '/content/SmartStitch/input [stitched]'
    with output:
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)
            print("✅ Pasta de saída limpa!")
        else:
            print("❌ Pasta de saída não existe!")

# --- 3. Botões ---
upload_button = widgets.Button(
    description="📤 Upload",
    button_style="info",
    tooltip="Enviar imagens para a pasta input"
)
upload_button.on_click(upload_images)

run_button = widgets.Button(
    description="▶️ Executar",
    button_style="success",
    tooltip="Executar SmartStitch com as configurações atuais"
)
run_button.on_click(run_smartstitch)

clear_input_button = widgets.Button(
    description="Lpar Entrada",
    button_style="danger",
    tooltip="Limpar a pasta de entrada"
)
clear_input_button.on_click(clear_input_folder)

clear_output_button = widgets.Button(
    description="Lpar Saida",
    button_style="danger",
    tooltip="Limpar a pasta de saida"
)
clear_output_button.on_click(clear_output_folder)

# --- 4. Montar Layout ---
ui = widgets.VBox([
    widgets.HTML("<h3>🧵 SmartStitch Console GUI</h3>"),
    widgets.HTML("<h4>⚙️ Configurações</h4>"),
    widgets.HBox([split_height, custom_width]),
    widgets.HBox([output_format, detection_type]),

    widgets.HTML("<h4>🎚️ Parâmetros Avançados</h4>"),
    sensitivity,
    quality,
    ignorable_pixels,
    scan_line,

    widgets.HTML("<h4>🎬 Execução</h4>"),
    widgets.HBox([upload_button, run_button, clear_input_button, clear_output_button]),
    zip_download,

    widgets.HTML("<h4>📊 Output</h4>"),
    output
])

# Criar a pasta input automaticamente ao inicializar
print("🔧 Inicializando SmartStitch GUI...")
create_input_folder()
#print("✅ Pasta input criada em /content/SmartStitch/input")

display(ui)